In [1]:
import json
import pandas as pd
import os
import gzip
import re

In [2]:
def read_data(files, path, keyword):
    data = []
    for file in files:
        if keyword in file:
            with gzip.open(path + file) as f:
                for line in f.readlines():
                    if 'Received prepared message'.encode('utf-8') in line \
                    or 'Receive announce message'.encode('utf-8') in line\
                    or 'Receive committed message'.encode('utf-8') in line:
                        data.append(json.loads(line))
    return data

In [3]:
def data_processing(data):
    df = pd.DataFrame(data)
    df['time'] = pd.to_datetime(df['time'], format = '%Y-%m-%dT%H:%M:%S.%f')
    df.sort_values(by=['time','message'], inplace = True)
    df.reset_index(inplace = True, drop = True)
    return df

In [4]:
def get_missing_msg_index(df):
    for name, group in df.groupby(df.index //3):
        if group.iloc[0]['message'] != '[OnAnnounce] Receive announce message':
            index = group.iloc[0].name
            print(index)
            break

In [8]:
log_dir = "/home/ubuntu/jupyter/logs/os/20/04/14/20:19:33/validator/tmp_log/log-20200405.194121/"
files = os.listdir(log_dir)
data = read_data(files, log_dir, '13.56.179.90')
df = data_processing(data)
df.groupby('message')['message'].count()

message
[OnPrepared] Received prepared message    6110
Name: message, dtype: int64

In [9]:
log_dir = "/home/ubuntu/jupyter/logs/os/20/04/14/19:35:10/validator/tmp_log/log-20200405.194121/"
files = os.listdir(log_dir)
data = read_data(files, log_dir, '13.56.179.90')
df = data_processing(data)
df.groupby('message')['message'].count()

message
[OnPrepared] Received prepared message    6110
Name: message, dtype: int64

In [10]:
log_dir = "/home/ubuntu/jupyter/logs/os/20/04/14/19:35:10/validator/tmp_log/log-20200405.194121/"
files = os.listdir(log_dir)
data = read_data(files, log_dir, '107.20.95.47')
df = data_processing(data)
df.groupby('message')['message'].count()

message
[OnPrepared] Received prepared message    6123
Name: message, dtype: int64